In [1]:
import os
import sys
import csv
import pickle
import numpy as np
import gurobipy as gp
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import loadmat
from time import perf_counter as timer
from concurrent.futures import ProcessPoolExecutor
from src import *

In [ ]:
data = Data()
thermal = Thermal()
renewable = Renewable()
demand = Demand()
commitment = Commitment()
data.load_thermal(thermal)
data.load_renewable_capacity(renewable)
data.load_renewable_generation(renewable)
data.load_demand(demand)
data.load_commitment_decision(commitment)
results = Results(thermal, renewable, commitment)   

In [ ]:
def parallel(idx_hour):
    solver = Solver(thermal, renewable, demand, commitment, results)
    solver.solve(idx_hour)
    
with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
    list(executor.map(parallel, range(8760)))
results.process_outputs()

Set parameter Username
Set parameter Username
Set parameter Username
Set parameter LicenseID to value 2659802
Set parameter LicenseID to value 2659802
Set parameter LicenseID to value 2659802
Academic license - for non-commercial use only - expires 2026-05-01
Academic license - for non-commercial use only - expires 2026-05-01
Academic license - for non-commercial use only - expires 2026-05-01
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter LicenseID to value 2659802
Set parameter LicenseID to value 2659802
Set parameter LicenseID to value 2659802
Academic license - for non-commercial use only - expires 2026-05-01
Academic license - for non-commercial use only - expires 2026-05-01
Academic license - for non-commercial use only - expires 2026-05-01
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
S

In [ ]:
# # sequential vs parallel sanity check

# # sequential
# for idx_hour in range(8760):
#     solver.solve(idx_hour)

# results.process_outputs()

# with open("sequential_results.pkl", "wb") as f:
#     pickle.dump(results, f)

# # parallel
# def parallel(idx_hour):
#     solver = Solver(thermal, renewable, demand, commitment, results)
#     solver.solve(idx_hour)

# with ProcessPoolExecutor(max_workers=20) as executor:
#     list(executor.map(parallel, range(8760)))

# results.process_outputs()

# with open("parallel_results.pkl", "wb") as f:
#     pickle.dump(results, f)

# with open("sequential_results.pkl", "rb") as f:
#     sequential_results = pickle.load(f)

# with open("parallel_results.pkl", "rb") as f:
#     parallel_results = pickle.load(f)

# print(np.all(np.array([np.all(results.smp == parallel_results.smp), np.all(results.cost_system == parallel_results.cost_system), np.all(results.p_thermal == parallel_results.p_thermal), np.all(results.p_solar == parallel_results.p_solar), np.all(results.p_wind == parallel_results.p_wind), np.all(results.p_hydro == parallel_results.p_hydro)])))
# print(np.all(np.array([np.all(results.smp == sequential_results.smp), np.all(results.cost_system == sequential_results.cost_system), np.all(results.p_thermal == sequential_results.p_thermal), np.all(results.p_solar == sequential_results.p_solar), np.all(results.p_wind == sequential_results.p_wind), np.all(results.p_hydro == sequential_results.p_hydro)])))
# print(np.all(np.array([np.all(parallel_results.smp == sequential_results.smp), np.all(parallel_results.cost_system == sequential_results.cost_system), np.all(parallel_results.p_thermal == sequential_results.p_thermal), np.all(parallel_results.p_solar == sequential_results.p_solar), np.all(parallel_results.p_wind == sequential_results.p_wind), np.all(parallel_results.p_hydro == sequential_results.p_hydro)])))

In [ ]:
# # preivous Data.load_renewable_capacity and Data.load_renewable_generation 

# def load_renewable_capacity(self, renewable):
#     renewable.count = 197
#     # 197 buses (IDK WHY the paper says 193 buses maybe version 2? below REG has 197 buses for all 365 csv so im going with 197)
#     # busid 119, 164, 185, 186 missing; solar capacity for these busid is assumed to be 0
#     with open(self.path_kpg / "renewables_capacity" /"solar_generators_2022.csv") as csvfile:
#         raw_solar = np.array([row for row in csv.reader(csvfile)])[1:, [0, 2]].astype(float)
#     renewable.solar_capacity = np.zeros(renewable.count)
#     renewable.solar_capacity[raw_solar[:, 0].astype(int) -1] = raw_solar[:, 1]

#     # same busid 119, 164, 185, 186 missing; the capacity for these busid is assumed to be 0
#     with open(self.path_kpg / "renewables_capacity" / "wind_generators_2022.csv") as csvfile:
#         raw_wind = np.array([row for row in csv.reader(csvfile)])[1:, [0, 2]].astype(float)
#     renewable.wind_capacity = np.zeros(renewable.count)
#     renewable.wind_capacity[raw_wind[:, 0].astype(int) -1] = raw_wind[:, 1]

#     # hydro capacity data is okay, full renewable 197 busid, all 0 pmin (4 empy pmin = 0) 
#     # also busid 197 has 400 max so 197 buses are probably intended
#     with open(self.path_kpg / "renewables_capacity" / "hydro_generators_2022.csv") as csvfile:
#         renewable.hydro_capacity = np.array([row for row in csv.reader(csvfile)])[1:, 2].astype(float)


# def load_renewable_generation(self, renewable):
#     solar_ratio, wind_ratio, hydro_ratio = np.empty((8760, renewable.count)), np.empty((8760, renewable.count)), np.empty((8760, renewable.count))

#     for idx_hour, file in zip(np.arange(0, 365 * 24, 24), self.get_path_files("renewables", "renewables")):
#         with open(file) as csvfile:
#             reader = csv.reader(csvfile)
#             next(reader)
#             raw = np.array([[float(cell) if cell else 0 for cell in row] for row in reader]) # empty cell correction
#             raw[:, [0, 1]] -= 1 # 0-indexing hour and busid

#             # please check KPG profile/renewables
#             # busid 153 is duplicate hourly (1h 1h 2h 2h ... 24h 24h)
#             # busid 151 is also missing... for all the 365 csv's
#             # which was why the length (24 * 197) was correct for each too
#             rawbusid153 = raw[(151 * 24):(153 * 24):2, :] # extracting hourly data from 2 * hourly duplicate in busid 153
#             raw[(150 * 24):(151 * 24), 1] = 150 # busid 151's data slided down from busid 152's data (0-indexed in code!) 
#             raw[(151 * 24):(152 * 24), :] = rawbusid153.copy() # busid 152's data is non duplicate busid 153's data 
#             raw[(151 * 24):(152 * 24), 1] = 151 # busid 152's data's busid set to 152 (0-indexed in code!)
#             raw[(152 * 24):(153 * 24), :] = rawbusid153.copy() # busid 152's data is non duplicate busid 153's data 
#             # raw[:, [0, 1]] += 1 # if you need this new corrected version as csv you can uncomment and save these 365 arrays
            
#             idx_hour += raw[:, 0].astype(int)
#             idx_bus = raw[:, 1].astype(int)
#             solar_ratio[idx_hour, idx_bus] = raw[:, 2]
#             wind_ratio[idx_hour, idx_bus] = raw[:, 3]
#             hydro_ratio[idx_hour, idx_bus] = raw[:, 4]

#     # 8760 hours, 197 buses
#     renewable.solar_generation = solar_ratio * renewable.solar_capacity
#     renewable.wind_generation = wind_ratio * renewable.wind_capacity
#     renewable.hydro_generation = hydro_ratio * renewable.hydro_capacity
#     # 8760 hours
#     renewable.total_solar_generation = renewable.solar_generation.sum(axis=1)
#     renewable.total_wind_generation = renewable.wind_generation.sum(axis=1)
#     renewable.total_hydro_generation = renewable.hydro_generation.sum(axis=1)
#     renewable.total_generation = renewable.total_solar_generation + renewable.total_wind_generation + renewable.total_hydro_generation

In [ ]:
# # cool plot
# fig, axes = plt.subplots(3, 1, figsize=(30, 30))

# for ax, name, reg_generation, reg_result in zip(
#     axes,
#     ["solar", "wind", "hydro"],
#     [renewable.solar_generation, renewable.wind_generation, renewable.hydro_generation],
#     [p_solar, p_wind, p_hydro]
# ):
        

#     idx_nonzero_reg = np.where(reg_generation[idx_hour] > 0 )[0]

#     # original reg
#     ax.bar(
#         range(len(idx_nonzero_reg)),
#         reg_generation[idx_hour][idx_nonzero_reg],
#         label="Original REG Amount (Derived from KPG193)",
#     )

#     # curtailed reg
#     ax.scatter(
#         range(len(idx_nonzero_reg)),
#         np.array(model.getAttr("X", reg_result.values()))[idx_nonzero_reg],
#         # facecolor=None, edgecolor="red", 
#         color="red", marker="x",
#         label="Curtailed REG Amount (Result of ED)",
#     )

#     ymax = ax.get_ylim()[1]

#     # curtailed reg marker
#     ax.vlines(
#         x=np.where(reg_generation[idx_hour][idx_nonzero_reg] > np.array(model.getAttr("X", reg_result.values()))[idx_nonzero_reg])[0],
#         ymin=0, ymax=ymax,
#         colors="red", linestyles="--",
#         label="Is this 'originally producing' REG bus curtailed?",
#     )

#     ax.set_ylim((0, ymax))
#     ax.set_xlim((-0.5, len(idx_nonzero_reg) - 0.5))
#     ax.set_title(f"Inspection of {name} Buses with (Originally) Nonzero Production\nhour in year = {idx_hour}\n(originallly infeasible without curtailment)", fontsize=15)
#     ax.set_xlabel(f"Nonzero generation {name} buses (not bus id)", fontsize=15)
#     ax.set_ylabel("Power generation (MWh)", fontsize=15)
#     ax.legend()

# plt.show(fig)